In [1]:
import pandas as pd
import numpy as np
import hnswlib
import time

from annoy import AnnoyIndex

In [2]:
hash_df = pd.read_pickle('arcface_files/hash_df.pkl')

In [3]:
hash_df

,ArcFace
Alex_Koji1,"[-0.07898863, 0.29689148, -2.5531418, -1.08932..."
Ariel_Guerreiro1,"[0.32658285, -0.7017476, -0.64553106, -0.57638..."
Ariel_Guerreiro2,"[0.5509473, -0.9909145, -0.34865752, -0.563928..."
Camila_Fonseca1,"[-1.5818309, -1.160022, 1.65914, 0.49970958, 2..."
Camila_Fonseca2,"[-2.0179617, 0.070794165, 2.3806381, -0.884002..."
Eduardo_Eiras1,"[-0.03154268, -1.23299, -0.6005332, 0.20772809..."
Eduardo_Eiras2,"[0.4015549, -1.7482035, -0.32626057, 0.1923889..."
Eduardo_Eiras3,"[0.09118195, -1.7747225, -0.42174166, 1.536756..."
Eduardo_Eiras4,"[-0.63092244, -1.3873752, -0.76119167, -0.2832..."
Eduardo_Eiras5,"[-0.090289675, -1.9531868, -0.14185497, 0.9069..."


## **Adicionando 100.000 novas representações aleatórias ao dataset**

In [4]:
data = hash_df['ArcFace'].to_numpy()
data = np.stack(data)

query = data[5, :]

data = np.delete(data, [5], axis=0)
random_data = np.random.normal(loc=0.0, scale=1.0, size=(int(1e5), data.shape[1]))
data = np.concatenate([data, random_data], axis=0)


data_labels = list(hash_df.index)
query_label = data_labels[5]
del data_labels[5]
random_labels = ['random_{}'.format(i) for i in range(int(1e5))]
data_labels = data_labels + random_labels
data_labels = np.asarray(data_labels)

# **Busca linear (exata)**

### Função de busca linear

In [5]:
def linear_search(query, data, data_labels, threshold=.35):
  
    query = np.expand_dims(query, axis=-1)
    data_dot_query = np.dot(data, query)

    diag = (data*data).sum(axis=-1)
    diag = np.expand_dims(diag, axis=-1)

    query_norm2 = np.dot(query.T, query)

    cosines = np.divide(data_dot_query, np.sqrt(query_norm2*diag))
    verif = cosines > threshold
    verif = verif.squeeze(axis=-1)

    return data_labels[np.argwhere(verif==True)]

### Salvando array de representações e *labels*

In [6]:
with open('search_files/search_data.npy', 'wb') as f:
    np.save(f, data)
    np.save(f, data_labels)

del data
del data_labels

### Carregando array de representações e labels

In [7]:
tic = time.time()
with open('search_files/search_data.npy', 'rb') as f:
    data = np.load(f)
    data_labels = np.load(f)
toc = time.time()

linear_load_time = toc - tic

### Realizando busca linear

In [8]:
tic = time.time()
linear_labels = linear_search(query, data, data_labels, threshold=.35)
toc = time.time()

linear_query_time = toc - tic

# **Hierarchical Navigable Small World (HNSW)**

Algoritmo de ANN baseado em grafos

**Estrutura de grafo em várias camadas**

<img src="imagens/hnsw_graph.PNG">

Malkov, Yuri et al. Efficient and robust approximate nearest neighbor search using Hierarchical Navigable Small World graphs (2016)

### Construindo, adicionando itens e salvando grafo

In [9]:
graph = hnswlib.Index(space='cosine', dim=512)
graph.init_index(max_elements=len(data), ef_construction=200, M=48)
graph.add_items(data, np.arange(len(data_labels)))
graph.set_ef(100)
graph.save_index('search_files/hnsw.bin')

del graph

### Carregando grafo

In [40]:
tic = time.time()
graph = hnswlib.Index(space='cosine', dim=512)
graph.load_index('search_files/hnsw.bin')
toc = time.time()

hnsw_load_time = toc - tic

### Realizando query

In [41]:
tic = time.time()
labels, distances = graph.knn_query(query, k=4)
toc = time.time()

In [42]:
hnsw_labels = data_labels[labels]
hnsw_query_time = toc - tic

In [13]:
del graph

# **Annoy**

Algoritmo de ANN baseado em árvores binárias

**Exemplo de uma das árvores binárias**

<img src="imagens/tree_annoy.PNG">

**Divisão de espaço bidimensional pela árvore**

<img src="imagens/space_annoy.PNG">

https://github.com/spotify/annoy

### Construindo, adicionando itens e salvando floresta

In [14]:
forest = AnnoyIndex(512, 'angular')

for i in range(len(data)):
    v = data[i, :]
    forest.add_item(i, v)

forest.build(n_trees=20)
forest.save('search_files/forest.ann')
del forest

### Carregando floresta

In [26]:
tic = time.time()
forest = AnnoyIndex(512, 'angular')
forest.load('search_files/forest.ann')
toc = time.time()

annoy_load_time = toc - tic

### Realizando query

In [31]:
tic = time.time()
labels = forest.get_nns_by_vector(query, n=4)
toc = time.time()

In [32]:
annoy_labels = data_labels[labels]
annoy_query_time = toc - tic

In [18]:
del forest

## **Resultados - Labels**

In [19]:
query_label

'Eduardo_Eiras1'

In [20]:
linear_labels

array([['Eduardo_Eiras2'],
       ['Eduardo_Eiras3'],
       ['Eduardo_Eiras4'],
       ['Eduardo_Eiras5']], dtype='<U20')

In [43]:
hnsw_labels

array([['Eduardo_Eiras3', 'Eduardo_Eiras2', 'Eduardo_Eiras5',
        'Eduardo_Eiras4']], dtype='<U20')

In [44]:
annoy_labels

array(['Eduardo_Eiras2', 'random_87000', 'random_18006', 'random_13390'],
      dtype='<U20')

## **Resultados - Tempo**

In [45]:
time_df = pd.DataFrame(columns=['Load', 'Query', 'Total'], index=['Busca Linear', 'HNSW', 'Annoy'])

In [46]:
load_list = [linear_load_time, hnsw_load_time, annoy_load_time]
query_list = [linear_query_time, hnsw_query_time, annoy_query_time]

time_df['Load'] = load_list
time_df['Query'] = query_list
time_df['Total'] = time_df['Load'] + time_df['Query']

In [47]:
time_df

,Load,Query,Total
Busca Linear,1.599056,0.259174,1.858229
HNSW,1.438586,0.000554,1.439140
Annoy,0.006971,0.001090,0.008061
